In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from icecube import astro, dataio,paraboloid
import histlite as hl
import csky as cy
from csky import hyp
import pandas as pd
from importlib import reload
import json
from astropy.time import Time
import glob
import tables

In [2]:
mc=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2016_20878_L5_MC.npy')

data_2011=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2011_L5_data.npy')
data_2012=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2012_L5_data.npy')
data_2013=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2013_L5_data.npy')
data_2014=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2014_L5_data.npy')
data_2015=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2015_L5_data.npy')
data_2016=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2016_L5_data.npy')
data_2017=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2017_L5_data.npy')
data_2018=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2018_L5_data.npy')
data_2019=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2019_L5_data.npy')
data_2020=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2020_L5_data.npy')
data_2021=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2021_L5_data.npy')
data_2022=np.load('/data/ana/PointSource/Galactic_Center/current/IC86_2022_L5_data.npy')

# data = np.concatenate((data_2011, data_2012, data_2013, data_2014, data_2015, data_2016, data_2017, 
#                       data_2018, data_2019, data_2020, data_2021, data_2022))

In [3]:
with open('/data/ana/PointSource/Galactic_Center/current/GRL/grl_2011_2022.json') as f:
    grl_data = f.read()
grl = json.loads(grl_data)

In [4]:
data = data_2012

In [5]:
cy.plotting.mrichman_mpl()
timer = cy.timing.Timer()
time = timer.time

GC_ra , GC_dec = astro.gal_to_equa(0., 0.)
sindec_uplim = np.sin(GC_dec + np.radians(10))
sindec_lolim = np.sin(GC_dec - np.radians(10))


LT_days = 0
for run in np.unique(data['run']):
    mask = (data['run'] == run)
    min_time = np.min(data[mask]['time'])
    max_time = np.max(data[mask]['time'])
    run_time = max_time - min_time
    #print(run_time)
    LT_days += run_time
LT = LT_days*86400
print(LT_days)

/mnt/ceph1-npx/user/xk35/software/external/csky/csky/plotting.py:92: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  r'\SetSymbolFont{operators}   {sans}{OT1}{cmss} {m}{n}'


337.6213908345817


In [6]:
start = []
stop = []
livetime = []
run = []
good_i3 = []
good_it = []
for item in list(grl.items())[1][1][:]:
    if(item['good_tstop']!=None):
        start.append(Time(item['good_tstart'], format='iso').mjd)
        stop.append(Time(item['good_tstop'], format='iso').mjd)
        run.append(item['run'])
        good_i3.append(item['good_i3'])
        good_it.append(item['good_it'])
    
livetime = np.array(stop)-np.array(start)

In [7]:
events = []
for r in run:
    mask = (data['run']==r)
    events.append(np.sum(mask))

In [8]:
dtype = [('run', int), ('events', int), ('start', float), ('stop', float),('livetime', float), ('good_i3', float), 
        ('good_it', float)]

grl_array = np.ones(len(run), dtype=dtype)
grl_array['run'] = run
grl_array['events'] = events
grl_array['start'] = start
grl_array['stop'] = stop
grl_array['livetime'] = livetime
grl_array['good_i3'] = good_i3
grl_array['good_it'] = good_it

In [9]:
mask_included_runs = (grl_array['events'] != 0)
grl_array = grl_array[mask_included_runs]
mask_good_runs = (grl_array['good_i3'] == 1) & (grl_array['good_it'] == 1)
if mask_good_runs.mean() != 1.0:
    print('!!!!!!!!! BAD RUNS ALART!!!!!!')
    print(mask_good_runs.mean())